# Readouts after a previous randomization

We've seen that randomization of lower layers has no effect on the final readout from a network, even though label information is in the CLS token. How quickly does information get read into the CLS tokens? Can I randomize and then get a novel readout from the next layer?

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.pylabtools import figsize

import seaborn as sns

import numpy as np
import pandas as pd

from tqdm import tqdm

In [2]:
from initialization import setup_model_ds_collator_images
from analysis import accuracy_post_randomization

model_name = "google/vit-base-patch16-224"
dataset_name = "zh-plus/tiny-imagenet"

W0112 15:35:39.965000 68099 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


import error: No module named 'triton'


In [3]:
model, dataset, _ = setup_model_ds_collator_images(model_name, dataset_name)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([200]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([200, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
template = 'vit.encoder.layer.{ind}'
outs = accuracy_post_randomization(model, dataset, template, max_ind=1, device='cpu')

Randomized: vit.encoder.layer.0
📕 Analysis: <function linear_probe_by_ridge_regression at 0x323cf6c00>
📄 Layer: vit.encoder.layer.0
{'device': 'cpu', 'batch_size': 64}


  0%|▏                                                                    | 2/625.0 [00:05<27:37,  2.66s/it]


KeyboardInterrupt: 